In [12]:
import json
import jsonlines
import os
import pandas as pd
import matplotlib.pyplot as plt
import cv2
dir_path = "../Data/annotations/"
ano = "open_images_train_v6_captions.jsonl"

In [21]:
image_id = []
captions = []
with jsonlines.open(dir_path+ano, 'r') as f:
    for line in f.iter():
        #if line['image_id'][0] == '0':
        image_id.append(line['image_id'])
        captions.append(line['caption'])


In [22]:
annotations = pd.DataFrame(image_id, columns=['image_id'])
annotations['caption'] = captions

In [7]:
annotations

,image_id,caption
0,fc06c1b59dd8ca6e,"We can see dolls,book and papers on bed and we..."
1,5833b3358fd949b5,In this picture I can see there is a airplane ...
2,67f1e4231b964a23,This looks like a black and white poster. I ca...
3,4839f5eac98771bf,In this image I can see on the left side it is...
4,64373c6a90e7b8dd,In this picture I can see group of people sitt...
...,...,...
507439,8cf26e4563f8c4bc,There is a lady wearing a tag and holding the ...
507440,c960dd40130f1d9a,In the image we can see there is a newspaper c...
507441,d4c2172f7f742d60,In the picture we can see table with drawers. ...
507442,60b2df9a3a7bd8d2,In this image we can see a girl. In the back t...


In [27]:
path = '../../disk/Data/open_image/'

img_paths = []
img_names = []
for folder, subfolders, filenames in os.walk(path):
    for img in filenames:
        img_paths.append(folder+'/'+img)
        img_names.append(img)
            
print('Images: ',len(img_names))


Images:  1743042


In [10]:
print(img_names[:5])

['0d602fc924a0a2a5.jpg', '0d5ad6dead4fd8d4.jpg', '0061e0fb55ff2d5c.jpg', '0abf55bf669af9c6.jpg', '0383b6b35606ce7e.jpg']


In [106]:
keyword = ['storm',
           'snowfall',
        ]
exclude_keyword = ['blazer','horse','candle',
                   'fireplace',
                   'lamp',
                   'photograph',
                   'ski','board','says','oven','cook', 'living', 'plug', 'hyr', 'food', 'young','kitchen', 'extinguisher',
                 'fireworks','vintage','department', 'kid', 'boy', 'grill','fire station','fox','engine', 'hyd','fires','jet','rocket']
k_image_id = []

In [19]:
annotations[annotations['caption'].str.contains('fire', case=False)]

,image_id,caption
8,40c66d14998ad13a,In this image I can see a woman wearing red co...
56,7164732223313801,This is a person standing. This looks like a f...
558,faec0d8ce842e27e,"This is an animated image , where there is a b..."
826,de51ffcf704a22ab,In this image we can see group of vehicles par...
1025,2ad42a74006edb6f,"In this picture I can see sculptures, there is..."
...,...,...
506198,70c61f3f25c634b8,"In this image, I can see few people standing a..."
506239,3f232d160c0d2f5d,There are three people standing. This person i...
506493,5380decd21c84b97,In this image there is a tent supported with m...
506789,6c03acbd11f6e2be,In this picture we can see a room with pillows...


In [107]:
find = annotations[annotations['caption'].str.contains('dododo', case=False)]

for i in keyword:
    find2 = annotations[annotations['caption'].str.contains(i, case=False)]
    find = pd.concat([find,find2], ignore_index=True)
    print(f'keyword : [{i}] // found: {len(find2)}')
print(f'final : {len(find)}')
## 결측지 유발 키워드 삭제
for i in exclude_keyword:
    origin_len = len(find)
    find = find[~find['caption'].str.contains(i, case=False)]
    print(f'excluded keyword:[{i}] found: {origin_len - len(find)}')
print(f'final : {len(find)}')
## 중복 이미지 삭제 (한 이미지에 대한 여러 caption)
find.drop_duplicates(['image_id'], inplace=True ,ignore_index = True, keep = 'first')
print(f'droped duplicate images from multiple captions, After dropped: {len(find)}')
## 인덱스 정렬
find.reset_index(inplace=True, drop=True)
#find

keyword : [storm] // found: 14
keyword : [snowfall] // found: 32
final : 46
excluded keyword:[blazer] found: 0
excluded keyword:[horse] found: 0
excluded keyword:[candle] found: 0
excluded keyword:[fireplace] found: 0
excluded keyword:[lamp] found: 2
excluded keyword:[photograph] found: 0
excluded keyword:[ski] found: 0
excluded keyword:[board] found: 1
excluded keyword:[says] found: 0
excluded keyword:[oven] found: 0
excluded keyword:[cook] found: 0
excluded keyword:[living] found: 0
excluded keyword:[plug] found: 0
excluded keyword:[hyr] found: 0
excluded keyword:[food] found: 0
excluded keyword:[young] found: 0
excluded keyword:[kitchen] found: 0
excluded keyword:[extinguisher] found: 0
excluded keyword:[fireworks] found: 0
excluded keyword:[vintage] found: 0
excluded keyword:[department] found: 0
excluded keyword:[kid] found: 0
excluded keyword:[boy] found: 0
excluded keyword:[grill] found: 0
excluded keyword:[fire station] found: 0
excluded keyword:[fox] found: 0
excluded keyword:

In [ ]:
find

In [109]:
find.sort_values(by=['image_id'],ascending=True, inplace=True)
find.reset_index(drop=True, inplace=True)
tmp = find['image_id'].value_counts().index.sort_values()

In [ ]:
cnt = 4
for i,key in enumerate(tmp):
    if cnt == 4:
        print('', end='\n\n\n\n')
        print(f'{(i+1)/4}.')
        cnt = 0
    print(f"'{key}'", end=' ')
    cnt+=1


In [ ]:
## show img status

back = 0
plt.figure(figsize=(16,500))
for i in range(len(find[back:])):
    plt.subplot(100,4,i+1)
    if i % 4 == 0:
        plt.title(f"{(i+1+back)/4}")
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    f_num = find['image_id'][i+back][0]
    
    path = '../../disk/Data/open_image/train_'+f_num+"/" + find['image_id'][i+back] +'.jpg'
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    plt.imshow(im_rgb, cmap=plt.cm.binary)
    plt.xlabel(find['caption'][i+back][:35] + '\n' + find['caption'][i+back][35:70] + '\n' 
               + find['caption'][i+back][70:105] + '\n image_id : ' + str(find['image_id'][i+back]) , loc='left', fontsize=10)
    #plt.legend(find['image_id'][i])
plt.show()

In [9]:
length = ['02fd19498de1e94a', '290d59dab457a369', '295b7671477f50f9', '2e8300d0999c77b8', 
'31e0a1de0ee988a8', '3f63a70a78520377', '4080d016eb273aae', '7e2449f75a0a0acc', 
'9c24a74e0769736c', '004676114a592ae3', '0b023637f13a726a', '0d4c9140e6b02485',
'0f44f7f46e94b877', '1c1396779a91706d', '3ce95037487f71aa', '76dbca04736cd468',
'8d21e84881a8c7cc', 'b6258d9fa4842a39', 'b9331416a89998cd', 'c71ed44f0ff40ab8',
'c9abbbebdf82a765', 'cdee239f0c42d32a', 'd78ae0ae07858c6d', 'd8f9319365edb45c', 
'fb8cb0720d3ed365', 'fc9e7c7089c900ca', 'fd0c8b2415814f84'
                   ]
print(len(length))
print(len(set(length)))

27
27


In [14]:
disaster_dataset = ['0218f76dff06752d', '02eab4f9ad6c0334', '0693141ad6ca7df3', '0bdfcf59dd6b55ce',
'0ff480920c9928c5', '101b499a52d7f46a', '112caf3e075e5394', '132eac17a04c4517',
'1550f14052bfec04', '183c1d7389c3b9ea', '1b24087aa0f9867b', '1df5638d77ff56cb',
'1e2a454911be0c5f', '2341349abfc26e90', '24f67cf7b161ea6b', '2b3651e8ba31ee65',
'2c67ed27579a6ea8', '345744962c7f4265', '34d50632542ecbb7', '35242c3f7a36c7bb', 
'36aee059e225bdaf', '3c6bf73ad74f1cd0', '3d789ac66fb6f955', '3f297fa8108336de',
'3f75d6405bd8a83f', '56234c17e1d60456', '5b612117e5778a4f', '5c7d9dc0a050e458',
'5d9d3da7f32e3ed0', '5dc549862bd6b8be', '6210ca5e86ae3d4e', '626d848b9b97ac60',
'62f96f9bddee4d68', '676ed076e09c9831', '705926a75728804d', '73ec259eab888d00',
'7633472d1ca39fe2', '78d846a0565d3bcd', '7b9ec91e0a4e22a9', '7e8eb56044f4b6fd',
'7eaf75eb4e77e2fb', '818a844fd465ac22', '83b32c162d09a4f8', '84948dc5e331047f',
'85f047ee06e75976', '88f9239e83cdb66d', '8b8fc6e31886cf55', '8e81cdd4461f27c4',
'9272694dbd156cd5', '9e6328c01768659b', 'a052c41f030af85b', 'a0cfb16260ece855',
'a2a59c80cef68850', 'a2e91881336cf349', 'a7a0b76240849487', 'a8f397a0f65a323c',
'ade8e68f664d382f', 'b06aa91a6083c7c2', 'b3fec6a2d8c7536a', 'b85303b3bdfed978',
'b8bc1b2847a360f1', 'be4d426c54e723b4', 'be5fb6daa44a3de5', 'bf8d2df0e0e9c22d',
'bfdd567c7a0f948b', 'c5b72236f87adafc', 'c745a2779725f898', 'c9503eb51eee2f65', 
'cf2221aa10e29a2c', 'e22d5041146ddcf1', 'ef7aeb8f945dbab2', 'f5d7ec658b470898',
'fd00a73e3f447913', '030f2923db6ddaf6', '073ff61ebe68b47d', '132dfe520c441034',
'159a00eee2eb6487', '17982de737021028', '1833cb7590c4a1e2', '1bdc8c1dc627a63e',
'20521f4ff0b2097b', '205beaeae430de7e', '207a67ddf458f89f', '2209a40e51151b46',
'249ac87de157a925', '250f902696a958fb', '258e7a7b282a55e3', '2875f5fac953d475',
'2b447b2177a103e9', '33ca13bf621e3611', '54703d04c4271a5a', '577e7ac0f47876b4',
'59eef36095c31cc8', '6d94b519e2bd3f38', '7a1d683f722af2d3', '86067733c4e30d83',
'87b2bdb4456388e1', '87c0da58e69c122c', '87b8f8da5c2257c7', '87cefa8085a5f80c',
'8defa4da1a395afb', '8e1dc5d47fffa64f', '9068ed5b7b32bf63', '923db4841e8217a1',
'990298e5cf07b5a6', 'a9883c4140659bbc', 'b25d2486ac41f4ad', 'b745f285c6773cc0',
'b91bb39004f7c275', 'c27f49edbd32f10e', 'caf67de657621c17', 'd479d07b6d9ca5d1',
'e103bc9c38834469', 'e4c1c08b0ff2fa78', 'e5aa9c1a2bf91af5', 'e8d034dcff14aca7',
'e8f4478fd0d9e3fb', 'e9d456b279d6c0f5', 'ea7c157a5a35c414', 'ec9ab2f37c531c02',
'ed389a47f3096a79', 'ed4796764c2d32d7', 'f1cc51ba23c2bc3d', 'f26cbe21df2edd66',
'f461fc8edaabe84e', 'f87e3c6eecd99585', 'f973f950eb7c8b39', 'fb9d113cdc3e70b3',
'fec82989d7ca0ee1', 'ffc4910c2c84a156','022b54e4e857ed33', '0de39b3791f2baa6', '15657f348712e734', '25099167e4022eba',
'26518efac4887b3c', '3797b764d267b064', '428509df148d1a27', '49aec59730a5a8aa',
'51255a7a1548080e', '52fde99971a5708c', '58d12e852edf194f', '5bbe14ccb14cd541', 
'64285be9eb0c2a46', '6c7b295c1bceb158', '6e16070bbbf48bde', '721af8090dfd27aa',
'874550cc18780297', '8a224c3b493fe762', '8cd19147d2c4f6b6', '91328c6645715517',
'a545a8059df4801f', 'b80089edc6bbc0ed', 'd0d6ed4f1ccd7041', 'd1db43f8dc0c3780', 
'e05bc81aa1d691d8', 'efd4e8e3e46acafc', 'f3c2b2265ca7bc45', 'fc365a2dd199e835',
'02c93939fdf8ecf8', '2775831e363c6a57', '62e9f35dd18a0e3a', '69c78f20a6a30ba4',
'743f72fd8aa0acb5', '75118fe313623da8', '76de10f902cfddf4', '79d0fd372493ef18',
'9389024025360785', '975ba43fb83523bc', 'a8271e8c2fa02700', 'be906c2c331923f0',
'bf63334e67a6c11d', 'c0dcc00f381bddcf', 'c214f06f43f83231', 'ce6f197f118b6c26',
'd507d1241f0c4f33', 'e96bd3fbab0d4357','00d425f0986ac579', '0128105921974df0', '01c4f65ea29a5632', '05b3130e553c6084',
'153e26395bea1973', '1656a46450a881f5', '17dba0da31436843', '189f6a62d7fa224c',
'2debae83fc6544cb', '4b2120ef0487e439', '502e1d8cb6b58911', '514efb5aee482d6f', 
'560e387f1b361a5c', '6a335f6ff3ffb6fd', 'bd62b456d26d69ea', 'd49698a15222f7fc',
'db1890e3518f1218', 'dc68faebac1558fa', 'dd4ceaf81843211a', 'e674841ecccb5a3c',
'fe2badae70e3f274', '004676114a592ae3', '07cb63de6ead014f', '0a49797a8f240259',
'1433fe91179ced97', '19c20d279292adfc', '212503981e231027', '2c7ed847f6aeca51',
'2f1ce4561995ca5f', '339765506757ee9d', '418617c896a9cfb8', '50883ded4a923ac0',
'50dbf6030e015e3d', '7311c4c9dbb5dcda', '75719757c412f287', '7cff597bf412c507',
'8092f610922fae95', '88c21cb844af2cb6', '88de8cedc1b4dd07', '9d2e07001a74e2f6',
'a9786487f36a069f', 'ba7f05226835eb4b', 'bb92b3c52eb018c8', 'c6dd39435d180e6f',
'd359a9f8483d3079', 'def24e1521d2c2db', 'eaadc79ed74b13f9', 'ec03b3d3af20f1c4','002a6cf0d401929c', '009c5fb34590f1ca', '00c5ae08ce541b77','02272e5568435fda'
'043ac011ff8afcf5', '04f67c62ee67f74b', '0589d82e0437d121', '0604cb9341631e15',
'08fef6f3a0e3bbcb', '2c57d43b16d35db1', '3bba5ee24ec0ee54', '413a993b9bb464ab', 
'522c4fb0babb19f1', '55baf0e2c4186a11', '5763316a89f6298d', '6122e9cf06bd61ce',
'6414f33c90e89519', '65258f8026ee8751', '678db2a731a13646', '6f7fc1281ee41e62',
'7658e9d7ea54e4f6', '82e3aea3423ca41a', '96d9477747f15afd', 'a0f87eeae39baf57',
'b74e7055559d0264', 'b8422e374555ab8a', 'b853a26933571d66','c478c70805c110f6',
'da3ce31355e6391f''d6b1f98d28bf6e8a', 'd759ef76b9095152', 'dd45f271c163fe8c',
'e7cef9a8ab2593a2', '0035c9cb0422e1b4', '0544618c0e2e0f86', '0622ef04fa206788',
'130fafe560e3bed3', '132176446f85f7f2', '20d69896f91b6839', '26e3a9cb71c0c50c',
'2b4e2b365696a3ce', '32ce6407b55b36a5', '32f367406aeab9c0', '3f1ec100e5841f58',
'43fb43c26c3e7981', '55b5f0b32b5025f2', '5c1b0c84307e4de7', '6641673d1914fb8f',
'685d73f92e75cdc4', '72e783657435987a', '7394dd66b6ff058d', '7782bdc79b6a0762',
'79d5a46f6417fa10', '7c17a76dbe912bf1', '883631022c1e06c0', '9b30616c2fe56171',
'a07b85e2b03909c1', 'a39bf6d02332b46d', 'ab84b2ffec20c9f4', 'adbf807d428052cc',
'b0c6dcd7300299d4', 'b550ad701c8fded8', 'b9a10fcfcc7d4e73', 'b9d8c94d56b04e63',
'bbfc8359da6971f6', 'bc1d16370ce2dd22', 'c4360ee77d317591', 'c9e540c74fae2648',
'ca326d6f01ed21d4', 'cbf5687e145dc19f', 'cc144958ab2a65ee', 'd482545a853f3d25',
'd91aa48d5dc31088', 'db5dce4627df903a', 'de2494dce555bec1', 'e49c8abd2d8eafe2',
'eaa910f252495a1c', 'ecfe8082bc0a55ca', 'f182d5c35d8e8342', 'f1d9076f1641f959',
'f1fad5ee685fc1b4', 'f307d5240a9b7ad9', 'f36101c507a1acfd', 'f55e1d7a19dfa563',
'f835d77270d53a85', 'fed7dc7536817fc0','02fd19498de1e94a', '290d59dab457a369', '295b7671477f50f9', '2e8300d0999c77b8', 
'31e0a1de0ee988a8', '3f63a70a78520377', '4080d016eb273aae', '7e2449f75a0a0acc', 
'9c24a74e0769736c', '004676114a592ae3', '0b023637f13a726a', '0d4c9140e6b02485',
'0f44f7f46e94b877', '1c1396779a91706d', '3ce95037487f71aa', '76dbca04736cd468',
'8d21e84881a8c7cc', 'b6258d9fa4842a39', 'b9331416a89998cd', 'c71ed44f0ff40ab8',
'c9abbbebdf82a765', 'cdee239f0c42d32a', 'd78ae0ae07858c6d', 'd8f9319365edb45c', 
'fb8cb0720d3ed365', 'fc9e7c7089c900ca', 'fd0c8b2415814f84'
                   ]
print(len(disaster_dataset))

335


#### 재난 데이터셋 이동

In [ ]:
import shutil

tmp = []
dest = '../Data/disaster_images/open_image_disaster/'
path = '../../disk/Data/open_image/'
for img in disaster_dataset:
    from_path = path+'train_'+img[0]+'/'+img+'.jpg'
    print(os.path.isfile(from_path))
    if os.path.isfile(from_path) == True:
        to_path = dest+img+'.jpg'
        
        #shutil.move(from_path,to_path)
        

#print(len(tmp))

In [23]:
captions = []
for img_id in disaster_dataset:
    cap = annotations[annotations['image_id'] == img_id]['caption'].value_counts().index
    for j in cap:
        #print(j)
        if len(j) < 250:
            captions.append(j.rstrip('\n').replace(",", ""))
print(captions)

['In the picture I can see two men are standing on the ground. These men are wearing helmets and holdings objects. In the background I can see trees the fire the grass plants the sky and some other objects on the ground.', 'There are plants walls windows chairs and other objects. These are damaged due to fire. And the background is dark in color.', 'In this image there are so many trees on the mountain at the back there is some fire and smoke.', 'In this image I can see a ladder which is in red color. Background I can see a house with fire and a building in white and brown color trees in green color and sky in white color.', 'This picture shows couple of men standing they wore sunglasses on their face and caps on their heads and we see trees and fire on the ground.', 'Here we can see trees fire and smoke. In the background there is sky.', 'At the bottom of this image I can see a campfire around this there are some stones. In the background there is a sea and I can see the trees.', 'In 

In [24]:
print(len(captions))

176
